In [3]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['font.family'] ='Malgun Gothic' 
plt.rcParams['axes.unicode_minus'] =False

warnings.filterwarnings('ignore')

# 1. 교통

In [4]:
d_master = pd.read_csv('교통/서울시 읍면동마스터 정보.csv',encoding='cp949')
d_subway = pd.read_csv('교통/서울시 행정동별 지하철 총 승차 승객수 정보.csv',encoding='cp949')

d_master.loc[d_master['행정동_ID']==1113066]
gu = d_master[['행정동_ID','자치구_명칭']]
merge = pd.merge(gu,d_subway,on='행정동_ID',how='right')
tf = merge[['자치구_명칭','지하철_승객_수_07시', '지하철_승객_수_08시', '지하철_승객_수_09시','지하철_승객_수_17시',
       '지하철_승객_수_18시', '지하철_승객_수_19시', '지하철_승객_수_20시']]

tf.columns=['시군구명','7시','8시','9시','17시','18시','19시','20시']
tf.fillna(0,axis=0,inplace=True)
tf[tf.columns[1:]] = tf[tf.columns[1:]].astype('int')
tf2 = tf.groupby(by='시군구명',as_index=False)[tf.columns[1:]].sum()
tf2.tail(1)

tf2['출근시간 지하철 승객수']= tf2['7시']+tf2['8시']+tf2['9시']
tf2['퇴근시간 지하철 승객수']= tf2['17시']+tf2['18시']+tf2['19시']+tf2['20시']
tf2
tf3=tf2[1:].reset_index(drop=True)
traffic = tf3[['시군구명','출근시간 지하철 승객수','퇴근시간 지하철 승객수']]
traffic
traffic.to_csv('traffic.csv',encoding='utf-8',index=False)

# 법정동코드

In [5]:
dong = pd.read_excel('KIKmix.20250513.xlsx')
dong1 = dong.loc[dong['시도명']=='서울특별시']
dong2 = dong1[2:].reset_index(drop=True)
dong3 = dong2[['시군구명','법정동코드']]
dong3

,시군구명,법정동코드
0,종로구,1111010100
1,종로구,1111010200
2,종로구,1111010300
3,종로구,1111010400
4,종로구,1111010500
...,...,...
764,강동구,1174010800
765,강동구,1174010800
766,강동구,1174010500
767,강동구,1174010600


In [6]:
dong2

,행정동코드,시도명,시군구명,읍면동명,법정동코드,동리명,생성일자,말소일자
0,1111051500,서울특별시,종로구,청운효자동,1111010100,청운동,20081101,NaN
1,1111051500,서울특별시,종로구,청운효자동,1111010200,신교동,20081101,NaN
2,1111051500,서울특별시,종로구,청운효자동,1111010300,궁정동,20081101,NaN
3,1111051500,서울특별시,종로구,청운효자동,1111010400,효자동,20081101,NaN
4,1111051500,서울특별시,종로구,청운효자동,1111010500,창성동,20081101,NaN
...,...,...,...,...,...,...,...,...
764,1174065000,서울특별시,강동구,성내제2동,1174010800,성내동,19880423,NaN
765,1174066000,서울특별시,강동구,성내제3동,1174010800,성내동,19880423,NaN
766,1174068500,서울특별시,강동구,길동,1174010500,길동,20080707,NaN
767,1174069000,서울특별시,강동구,둔촌제1동,1174010600,둔촌동,19880423,NaN


In [7]:
d4 = dong3.drop_duplicates(subset='법정동코드')
d4

,시군구명,법정동코드
0,종로구,1111010100
1,종로구,1111010200
2,종로구,1111010300
3,종로구,1111010400
4,종로구,1111010500
...,...,...
757,강동구,1174010700
760,강동구,1174010900
763,강동구,1174010800
766,강동구,1174010500


# 2. 범죄

## 1. 범죄발생

In [25]:
crime = pd.read_csv('범죄\경찰청_범죄 발생 지역별 통계_20231231.csv',encoding='CP949')
crime
cols = [col for col in crime.columns if ("서울" in col) or (col=='범죄대분류') or (col=='범죄중분류')]
crime2 = crime[cols]
# crime2['범죄분류']=crime2['범죄대분류'] +'_'+ crime2['범죄중분류']
# c3 = crime2.drop(['범죄대분류','범죄중분류'],axis=1)
c4=crime2[['범죄중분류','서울종로구', '서울중구', '서울용산구', '서울성동구', '서울광진구', '서울동대문구', '서울중랑구', '서울성북구',
       '서울강북구', '서울도봉구', '서울노원구', '서울은평구', '서울서대문구', '서울마포구', '서울양천구', '서울강서구',
       '서울구로구', '서울금천구', '서울영등포구', '서울동작구', '서울관악구', '서울서초구', '서울강남구', '서울송파구',
       '서울강동구']]
c4
c5 = pd.melt(c4,id_vars=['범죄중분류'],var_name='지역',value_name='발생건수')
c5
# 피벗 cols='범죄분류',row='지역',value='발생건수'

c6=pd.pivot_table(c5, index='지역',columns='범죄중분류',values='발생건수')
c7=c6.reset_index(drop=False)
c7['지역']=c7['지역'].str.replace('서울','')
c7
c8=c7[['지역','강간','강제추행','기타 강간 강제추행등','살인미수등','살인기수','성풍속범죄','유사강간']]
c8.index.name=None
c8
crime = c8.rename(columns={'지역':'시군구명'})

cols = ['강간', '강제추행', '기타 강간 강제추행등', '살인미수등', '살인기수', '성풍속범죄', '유사강간']
crime[cols]=crime[cols].astype(int)
crime.columns.name=None
crime.to_csv('crime.csv',encoding='cp949',index=False)
crime.tail(1)

,시군구명,강간,강제추행,기타 강간 강제추행등,살인미수등,살인기수,성풍속범죄,유사강간
24,중랑구,40,118,0,2,4,122,9


In [9]:
# merge2 = pd.merge(c9,d4,on='시군구명',how='inner')
# merge3=merge2[['시군구명','동리명', '법정동코드', '강간', '강제추행', '기타 강간 강제추행등', '살인미수등', '살인기수', '성풍속범죄', '유사강간']]
# merge3

# 3.안전

## 1. 유흥

In [10]:
# 유흥주점
df = pd.read_csv("범죄\서울시 유흥주점영업 인허가 정보.csv",encoding='cp949')
df
df2=df.loc[df['영업상태명']!='폐업',['지번주소','사업장명','최종수정일자','데이터갱신일자','업태구분명','좌표정보(X)','좌표정보(Y)','영업장주변구분명']]
df2
df2['지번주소']=df2['지번주소'].str.strip('서울특별시')
df2[['시군구명', '동리명']] = df2['지번주소'].str.extract(r'(\S+구)\s+(\S+동)')
df2
df3=df2.groupby(by=['시군구명','동리명','업태구분명'],as_index=False)['사업장명'].count()
df3
df4 = pd.pivot_table(df3, index='시군구명', columns='업태구분명', values='사업장명', aggfunc='sum')
df4.reset_index(drop=False, inplace=True)
df4

df4['클럽']=df4[['고고(디스코)클럽', '관광호텔나이트(디스코)', '관광호텔나이트(카바레)', '노래클럽','카바레']].sum(axis=1)
df4['룸살롱']=df4[['룸살롱', '비어(바)살롱', '스텐드바', '요정',]].sum(axis=1)
df4['주점']=df4[['간이주점','극장식당','기타']].sum(axis=1)
df4.tail()

df5 = df4[['시군구명','클럽','주점','룸살롱']]
df5.columns.name=None
cols=['클럽','주점','룸살롱']
df5[cols]=df5[cols].astype('int')
df5.tail(1)

,시군구명,클럽,주점,룸살롱
24,중랑구,3,0,14


In [11]:
# d_idx = merge3.loc[merge3['동리명'].str.contains('구')].index
# d_idx
# merge4 = merge3.drop(d_idx,axis=0)
# merge4.reset_index(drop=True,inplace=True)
# merge4

In [12]:
# cols=['강간', '강제추행', '기타 강간 강제추행등', '살인미수등', '살인기수',
#        '성풍속범죄', '유사강간']
# merge4[cols]=merge4[cols].astype('int')
# merge4


In [13]:
# merge6 = merge5.fillna(0)
# cols=['클럽','주점','룸살롱']
# merge6[cols]=merge6[cols].astype('int')
# merge6

## 2. 가로등 개수

In [14]:
# 가로등 개수
import geopandas as gpd
from shapely.geometry import Point

# 1. 법정동 경계 파일 불러오기
gu = gpd.read_file("LARD_ADM_SECT_SGG_11_202505.shp",encoding='cp949')

# 2. 좌표계 통일 (법정동 shp가 보통 EPSG:5179 또는 EPSG:4326)
gu = gu.to_crs(epsg=4326)
gu['SGG_NM']=gu['SGG_NM'].str.strip('서울특별시 ')
gu2 = gu[['SGG_NM','geometry']]
gu2.columns=['시군구명','geometry']
gu2
# 3. 가로등 데이터 불러오기
df = pd.read_csv('범죄/서울시 가로등 위치 정보 (2).csv',encoding='cp949')
df
# 4. GeoDataFrame으로 변환
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['경도'], df['위도']), crs="EPSG:4326")

# 5. 공간조인: 가로등 위치와 법정동 경계 매핑
merged = gpd.sjoin(gdf, gu2, how='left', predicate='intersects')
merged
# # 6. 법정동별 개수 세기
streetlamp = merged.groupby(by='시군구명',as_index=False)['관리번호'].count()
streetlamp.columns=['시군구명','가로등수']
streetlamp.tail(1)

merge = pd.merge(df5,streetlamp,on='시군구명',how='outer')
merge

,시군구명,클럽,주점,룸살롱,가로등수
0,강남구,22.0,3.0,144.0,1785.0
1,강동구,5.0,6.0,88.0,861.0
2,강북구,8.0,14.0,47.0,NaN
3,강서구,15.0,6.0,108.0,1209.0
4,관악구,15.0,11.0,170.0,96.0
5,광진구,1.0,0.0,9.0,1313.0
6,구로구,7.0,4.0,21.0,314.0
7,금천구,1.0,3.0,40.0,35.0
8,노원구,3.0,1.0,17.0,645.0
9,대문구,NaN,NaN,NaN,381.0


## 3. 소방서

### - 각 동의 인구당 소방시설 개수 집계해서 대응 인프라 수준 파악하겠음

In [15]:
df= pd.read_excel('범죄/서울시 소방서,안전센터,구조대 위치정보.xlsx')
df
# 2. GeoDataFrame으로 변환 (현재 좌표계는 EPSG:5179)
gdf = gpd.GeoDataFrame(df, 
                       geometry=gpd.points_from_xy(df['X좌표'], df['Y좌표']),
                       crs='EPSG:5186')

# 3. 좌표계를 EPSG:4326으로 변환 (법정동 shp와 맞추기 위해)
gdf = gdf.to_crs(epsg=4326)
gdf
# 4. 법정동 shp 파일 불러오기 및 좌표계 확인/변환
gu = gpd.read_file("LARD_ADM_SECT_SGG_11_202505.shp",encoding='cp949')

# 2. 좌표계 통일 (법정동 shp가 보통 EPSG:5179 또는 EPSG:4326)
gu = gu.to_crs(epsg=4326)
gu['SGG_NM']=gu['SGG_NM'].str.strip('서울특별시 ')
gu2 = gu[['SGG_NM','geometry']]
gu2.columns=['시군구명','geometry']
gu2

# 5. 공간조인 (포인트가 동 경계 안에 있을 때)
merged = gpd.sjoin(gdf, gu2, how='left', predicate='within')
merged
# 6. 법정동별 개수 세기
safer = merged.groupby(by='시군구명',as_index=False)['유형구분명'].count()
safer.rename(columns={'유형구분명':'소방서수'},inplace=True)

merge = pd.merge(safer,merge,on='시군구명',how='outer')
merge

,시군구명,소방서수,클럽,주점,룸살롱,가로등수
0,강남구,8.0,22.0,3.0,144.0,1785.0
1,강동구,9.0,5.0,6.0,88.0,861.0
2,강북구,6.0,8.0,14.0,47.0,NaN
3,강서구,9.0,15.0,6.0,108.0,1209.0
4,관악구,7.0,15.0,11.0,170.0,96.0
5,광진구,6.0,1.0,0.0,9.0,1313.0
6,구로구,8.0,7.0,4.0,21.0,314.0
7,금천구,4.0,1.0,3.0,40.0,35.0
8,노원구,7.0,3.0,1.0,17.0,645.0
9,대문구,6.0,NaN,NaN,NaN,381.0


In [16]:
bell = pd.read_excel('범죄\서울특별시_안전비상벨위치정보(2025.03.07.기준) (3).xlsx')
bell['설치위치'].value_counts()

설치위치
일원터널 내 보행로            1
연희지하보차도 내 보행로         1
대방지하보차도 내 보행로         1
잠수교남측지하보차도 내 보행로      1
방학지하차도 내 보행로          1
월계지하차도 내 보행로          1
창동지하차도 내 보행로          1
녹천지하차도 내 보행로          1
능산길 지하보차도 내 보행로       1
답십리굴다리지하보차도 내 보행로     1
군자교동측지하차도 내 보행로       1
신내지하보차도 내 보행로         1
마장지하보차도 내 보행로         1
구로역지하차도 내 보행로         1
가마산지하보차도 내 보행로        1
신도림지하보차도 내 보행로        1
목동교서측지하보차도 내 보행로      1
오목교서측지하보차도 내 보행로      1
서울교접속남단지하보차도 내 보행로    1
국회앞지하차도 내 보행로         1
아차산지하차도 내 보행로         1
서리풀터널 내 보행로           1
율곡터널 내 보행로            1
영등포지하보차도 내 보행로        1
증산지하보차도 내 보행로         1
매봉터널 내 보행로            1
잠수교북단지하보차도 내 보행로      1
금화터널 내 보행로            1
월드컵터널 내 보행로           1
호암2터널 내 보행로           1
상도터널 내 보행로            1
난곡터널 내 보행로            1
산복터널 내 보행로            1
호암1터널 내 보행로           1
북악터널 내 보행로            1
구기터널 내 보행로            1
자하문터널 내 보행로           1
공릉터널 내 보행로            1
솔샘터널 내 보행로            1
사직터널 내 보행로            1
금호터널 내 보행로            1
무지개터널 내 보행로

### 4. cctv

#### 시군구명 매핑...

In [26]:
cctv = pd.read_excel('범죄\서울시 자치구 (목적별) CCTV 설치현황_241231 (2).xlsx')
cctv2 = cctv[1:]
cctv2.head()
cctv3 = cctv2.drop('Unnamed: 0',axis=1).reset_index(drop=True)
cctv3.loc[1,'Unnamed: 1']='시군구명'
cctv3.loc[1,'Unnamed: 2']='CCTV 총계'
cctv3.columns = cctv3.loc[1]
cctv4 = cctv3[['시군구명','CCTV 총계']][3:]
cctv4.columns.name=None

cctv5 = cctv4.reset_index(drop=True).dropna(axis=0)
cctv5.loc[cctv5['시군구명']=='동대문','시군구명']='동대문구'
cctv5
safe = pd.merge(cctv5,merge,on='시군구명',how='outer')
safe
safe.to_csv('safe.csv',encoding='cp949',index=False)

In [ ]:
safe2 = safe.copy()


# 합산할 컬럼
sum_cols = ['CCTV 총계','소방서수','클럽','주점','룸살롱','가로등수']

# 숫자화(문자/NaN 대비)
safe2.fillna(0,axis=1,inplace=True)
safe2[sum_cols] = safe2[sum_cols].astype(int)


# 표준명칭 ← 변형들
alias_map = {
    '서대문구': ['서대문구', '서대문', '대문구'],
    '서초구':   ['서초구', '초구'],
    '영등포구': ['영등포구', '영등포'],
}

for canon, names in alias_map.items():
    # 합계 구하기
    s = safe2.loc[safe2['시군구명'].isin(names), sum_cols].sum(axis=0)

    # 표준 행 존재 여부
    has_canon = (safe2['시군구명'] == canon).any()
    if has_canon:
        safe2.loc[safe2['시군구명'] == canon, sum_cols] = s.values
    else:
        # 표준 행 없으면 새로 추가
        new_row = {col: None for col in safe2.columns}
        new_row['시군구명'] = canon
        for c in sum_cols:
            new_row[c] = s[c]
        safe2 = pd.concat([safe2, pd.DataFrame([new_row])], ignore_index=True)

    # 표준명을 제외한 별칭 행 드롭(원하면 주석 해제)
    to_drop = [n for n in names if n != canon]
    safe2 = safe2.loc[~safe2['시군구명'].isin(to_drop)].reset_index(drop=True)

# 확인
safe2.loc[safe2['시군구명'].isin(alias_map.keys()), ['시군구명', *sum_cols]]
safe2.to_csv('safe.csv',encoding='cp949',index=False)

# 4. 소음

In [ ]:
# 23년도 1,2,3,4분기 데이터
noise = pd.read_csv('소음\소음진동민원+현황_20250408223939.csv')
noise.replace('-',0,inplace=True)

noise23 = noise[['자치구별(2)','2023','2023.1']]
noise23.columns=noise23.loc[0].values
noise23=noise23[3:].reset_index(drop=True)

noise23.rename(columns={'자치구별(2)':'시군구명'},inplace=True)
noise23
noise23.to_csv('noise.csv',encoding='cp949')

,시군구명,환경관련 전체민원,소음진동 민원
0,종로구,2512,2200
1,중구,1678,1223
2,용산구,3614,3175
3,성동구,2691,2574
4,광진구,1516,1374
5,동대문구,3537,3437
6,중랑구,3303,3003
7,성북구,3056,2775
8,강북구,1398,1192
9,도봉구,1293,1153


# 5. 인구(2019~2024)

In [19]:
df = pd.read_csv('인구밀도/서울시 상권분석서비스(길단위인구-행정동).csv',encoding='cp949')
df.head(5)
df2.info()
df2=df[['기준_년분기_코드', '행정동_코드_명', '총_유동인구_수', '남성_유동인구_수', '여성_유동인구_수',
       '연령대_10_유동인구_수', '연령대_20_유동인구_수', '연령대_30_유동인구_수', '연령대_40_유동인구_수',
       '연령대_50_유동인구_수', '연령대_60_이상_유동인구_수']]
import re

df2['행정동_코드_명'] = df2['행정동_코드_명'].str.replace(r'[^\uAC00-\uD7A3]', '', regex=True)

df2.rename(columns={'행정동_코드_명':'동리명'},inplace=True)

df2['기준_년분기_코드'] = df2['기준_년분기_코드'].astype(str).str.slice(0, -1)
df2.columns=['연도','동리명','총유동인구','남성유동인구','여성유동인구','10대','20대','30대','40대','50대','60대이상']
df3 = df2.groupby(by=['연도','동리명'],as_index=False)[['총유동인구','남성유동인구','여성유동인구','10대','20대','30대','40대','50대','60대이상']].sum()
df3.drop('연도',axis=1,inplace=True)
df3
dong2.loc[dong2['읍면동명'].str.contains('중계본', na=False)]
dong3=dong2[['시군구명','동리명']]
dong3
dong2['동리명']
merge=pd.merge(dong3,df3,on='동리명',how='inner')
people = merge.groupby(by='시군구명',as_index=False)[['총유동인구','남성유동인구','여성유동인구','10대','20대','30대','40대','50대','60대이상']].sum()
people

<class 'pandas.core.frame.DataFrame'>
Index: 1751 entries, 0 to 4838
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   지번주소      1750 non-null   object 
 1   사업장명      1751 non-null   object 
 2   최종수정일자    1751 non-null   object 
 3   데이터갱신일자   1751 non-null   object 
 4   업태구분명     1751 non-null   object 
 5   좌표정보(X)   1736 non-null   float64
 6   좌표정보(Y)   1736 non-null   float64
 7   영업장주변구분명  697 non-null    object 
 8   시군구명      1595 non-null   object 
 9   동리명       1595 non-null   object 
dtypes: float64(2), object(8)
memory usage: 150.5+ KB


,시군구명,총유동인구,남성유동인구,여성유동인구,10대,20대,30대,40대,50대,60대이상
0,강남구,9249992749,4516747755,4733245027,1500838229,1498477319,1770852777,1738235867,1213854803,1527733876
1,강동구,6365187945,2982893421,3382294613,1051324050,773816073,995133849,1024037496,970880811,1549995826
2,강북구,4904290145,2262869790,2641420389,597965209,782212324,644609076,712495144,791489909,1375518531
3,강서구,12252961385,5729393970,6523567528,1746882193,1761373327,2258026438,1939052250,1696897483,2850729941
4,관악구,2747851268,1406755372,1341095871,168641375,940920958,634758406,346179824,267151196,390199553
5,광진구,6853164613,3192349858,3660814715,998324224,1153954673,1148834290,1054287509,1036192132,1461571720
6,구로구,6176441538,3006059866,3170381723,795847590,786259486,1052978996,1057644059,1000147041,1483564364
7,금천구,3684666298,1784045649,1900620688,458391309,495617835,611804028,577150732,622252693,919449941
8,노원구,13204181026,6029643966,7174537166,2280820024,1742117629,1611096800,1942496027,2113137694,3514513055
9,도봉구,6300691794,2900971455,3399720326,987919835,779505222,830658797,931208270,1011743801,1759656061


# 6. 소득(19~24)

In [20]:
df2 = pd.read_csv('서울시 상권분석서비스(소득소비-행정동).csv',encoding='cp949')
df2['행정동_코드_명'] = df2['행정동_코드_명'].str.replace(r'[^\uAC00-\uD7A3]', '', regex=True)

df2.rename(columns={'행정동_코드_명':'동리명'},inplace=True)

df2['기준_년분기_코드'] = df2['기준_년분기_코드'].astype(str).str.slice(0, -1)
df3=df2[['기준_년분기_코드',  '동리명', '월_평균_소득_금액',  '지출_총금액',
       '식료품_지출_총금액', '의류_신발_지출_총금액', '생활용품_지출_총금액', '의료비_지출_총금액', '교통_지출_총금액',
       '교육_지출_총금액', '유흥_지출_총금액', '여가_문화_지출_총금액', '기타_지출_총금액', '음식_지출_총금액']]
df3.columns = ['연도','동리명','월평균소득','총지출','식료품비','의류비','의료비','교육비','교통비','생활용품비','여가비','유흥비','식비','기타']

cols=df3.columns[2:]
df4=df3.groupby(by=['연도','동리명'],as_index=False)[cols].sum()
df4


merge=pd.merge(dong3,df4,on='동리명',how='inner')
income = merge.groupby(by='시군구명',as_index=False)[cols].sum()
income.head()

,시군구명,월평균소득,총지출,식료품비,의류비,의료비,교육비,교통비,생활용품비,여가비,유흥비,식비,기타
0,강남구,7123406280,110672997496000,2278658302000,1386070858000,535188691000,5979127577000,2040529029000,2216831689000,529206260000,53284717809000,38462696221000,3959971060000
1,강동구,3452820729,9613279290000,1111966624000,128671955000,92163422000,935264160000,190619698000,273119235000,143695402000,5396806085000,190592679000,1150380030000
2,강북구,1701740568,3079539728000,682014828000,105745180000,52218227000,497141912000,242167935000,45521370000,201210362000,223444736000,146140457000,883934721000
3,강서구,6105411975,16672848758000,2609183722000,437697939000,197029672000,3455117418000,2084582933000,356866278000,537341166000,2374113062000,1612976621000,3007939947000
4,관악구,727379907,2241187003000,487062983000,51483035000,31773904000,406016078000,20393284000,19961038000,186772695000,220791888000,138410041000,678522057000


# 7. 편의시설

In [21]:
df1 = pd.read_csv('편의시설/서울시 문화공간 정보.csv',encoding='cp949')
df1.tail(1)

# df1.loc[(df1['주소'].str.contains('강남구')) & (df1['주제분류']=='기타'),['주제분류','문화시설명']]

# df1.loc[df1['주제분류']=='기타','문화시설명']
df2 = df1[['주제분류','주소','문화시설명']]

df2['주소'] = df2['주소'].str.extract(r'(\S+구)\s')
# pd.pivot_table(df3, index='시군구명', columns='업태구분명', values='사업장명', aggfunc='sum')
df3=pd.pivot_table(df2,index='주소',columns='주제분류',values='문화시설명',aggfunc='count')
# df2.groupby(by='주소',as_index=False)['주제분류'].count()
df3.fillna(0,axis=0,inplace=True)
df4 = df3[df3.columns].astype('int')

df4.columns.name=None
df4.reset_index(drop=False,inplace=True)
df4

,주소,공연장,기타,도서관,문화예술회관,문화원,미술관/갤러리,박물관/기념관
0,강남구,20,13,18,0,0,29,9
1,강동구,1,3,7,1,0,0,1
2,강북구,2,1,5,1,0,2,2
3,강서구,2,3,3,1,1,3,2
4,관악구,0,6,3,1,0,2,2
5,광진구,5,3,7,1,1,2,3
6,구로구,4,5,11,1,1,2,2
7,금천구,2,5,4,1,1,0,0
8,노원구,2,5,6,1,2,5,2
9,도봉구,1,7,8,0,1,0,5


In [22]:
## 구별로 문화시설명 정리
# df3=pd.pivot(data=df2,index='주제분류',columns='문화시설명',values='주소')
# df3
df3 = pd.pivot_table(df2, index='주소', columns='주제분류', values='문화시설명', aggfunc=lambda x: ', '.join(sorted(set(x))))
df3.columns.name=None
df4 = df3.reset_index(drop=False)
df4.fillna(0,inplace=True)
df4

,주소,공연장,기타,도서관,문화예술회관,문화원,미술관/갤러리,박물관/기념관
0,강남구,"GB 가빈아트홀, KT&G 상상마당 대치아트홀, 강남씨어터, 거암아트홀, 광림아트센...","SETEC전시장(서울무역전시장), SPACE 1257, 강남구민회관, 강남스포츠문화...","강남구립 개포하늘꿈도서관, 강남구립논현도서관, 강남구립대치도서관, 강남구립도곡정보문...",0,0,"ADM 갤러리, K현대미술관, VIVIAN CHOI GALLERY, 갤러리 JJ, ...","경운박물관, 국립관세박물관, 도산안창호 기념관, 선정릉, 세계골프역사박물관, 코리아..."
1,강동구,강동어린이회관아이누리홀,"강동구민회관, 광진교 8번가, 온조대왕문화체육관","강동구립성내도서관, 강동구립암사도서관, 강동구립천호도서관, 강동구립해공도서관, 서울...",강동아트센터,0,0,암사동선사유적박물관
2,강북구,"북서울꿈의숲아트센터, 북서울꿈의숲아트센터 퍼포먼스홀",서울문화예술교육센터 강북,"강북문화정보도서관, 강북청소년문화정보도서관, 솔샘문화정보도서관, 송중문화정보도서관,...",강북문화예술회관,0,"북서울꿈의숲 상상톡톡미술관, 북서울꿈의숲아트센터 드림갤러리","근현대사기념관, 봉황각"
3,강서구,"LG아트센터 서울, 스카이아트홀","강서구민회관, 국립국어원 우리말 꿈터, 서울퓨처랩","강서구립길꽃어린이도서관, 서울특별시교육청 강서도서관, 아리향기 작은도서관",강서아트리움,강서문화원,"겸재정선미술관, 스페이스K 서울, 아울 갤러리","기생충박물관, 허준박물관"
4,관악구,0,"관악청년청, 관악청소년센터, 관천로 문화플랫폼 S1472, 극장작당모의, 서울시민대...","관악중앙도서관, 글빛정보도서관, 낙성대공원도서관",관악아트홀,0,"서울대학교 미술관, 서울시립 남서울미술관","서울대학교박물관, 호림박물관(신림본관)"
5,광진구,"건국대학교 새천년관대공연장, 광진어린이공연장, 세종대학교 대양홀, 예스24 라이브홀...","구의아리수문화유산, 서울거리예술창작센터, 한강이야기전시관","광진정보도서관, 구의제3동도서관, 군자동도서관, 아차산 숲속도서관, 자양4동도서관,...",나루아트센터,몽골 울란바타르 문화진흥원,"삼원갤러리, 세종뮤지엄갤러리","건국대학교 박물관, 서울상상나라, 세종대학교 박물관"
6,구로구,"구로 꿈나무극장, 구로창의아트홀, 신도림 오페라하우스, 오류아트홀","구로구민회관, 서울시50플러스 남부캠퍼스, 서울아트책보고, 신도림 문화공간 다락, ...","개봉도서관, 구로구립궁동어린이도서관, 구로구립글마루 한옥어린이도서관, 구로구립꿈나무...",구로아트밸리예술극장,구로문화원,"갤러리 구루지, 구캔 갤러리 : 구독하는 캔버스","G밸리산업박물관, 평강성서유물박물관"
7,금천구,"금천뮤지컬센터, 문화정원아트홀","금나래갤러리, 금천마을활력소 어울샘, 금천예술공장, 만천명월예술인家, 하라스튜디오","금천구립가산도서관, 금천구립금나래도서관, 금천구립독산도서관, 금천구립시흥도서관",금나래아트홀,금천문화원,0,0
8,노원구,"광운대학교동해문화예술관, 노원어린이극장","공릉청소년문화정보센터, 노원수학문화관, 문화공간 정담, 문화살롱 5120, 서울시립과학관","노원어린이도서관, 노원중앙도서관, 불암도서관, 상계숲속작은도서관, 서울특별시교육청 ...",노원문화예술회관,"노원문화원, 노원문화의집","경춘선숲길 갤러리, 상계예술마당, 서울과학기술대학교 미술관, 서울시립 북서울미술관,...","서울생활사박물관, 육군박물관"
9,도봉구,극단 대학로극장,"도봉구민회관, 마을극장 흰고무신, 서울로봇인공지능과학관, 씨알방학간, 창동문화체육센...","김근태기념도서관, 도봉기적의도서관, 도봉문화정보도서관, 도봉아이나라도서관, 서울특별...",0,도봉문화원,0,"간송옛집, 김수영문학관, 덕성여자대학교 박물관, 둘리뮤지엄, 함석헌기념관"


In [23]:
help(pd.pivot)

Help on function pivot in module pandas.core.reshape.pivot:

pivot(data: 'DataFrame', *, columns: 'IndexLabel', index: 'IndexLabel | lib.NoDefault' = <no_default>, values: 'IndexLabel | lib.NoDefault' = <no_default>) -> 'DataFrame'
    Return reshaped DataFrame organized by given index / column values.

    Reshape data (produce a "pivot" table) based on column values. Uses
    unique values from specified `index` / `columns` to form axes of the
    resulting DataFrame. This function does not support data
    aggregation, multiple values will result in a MultiIndex in the
    columns. See the :ref:`User Guide <reshaping>` for more on reshaping.

    Parameters
    ----------
    data : DataFrame
    columns : str or object or a list of str
        Column to use to make new frame's columns.
    index : str or object or a list of str, optional
        Column to use to make new frame's index. If not given, uses existing index.
    values : str, object or a list of the previous, optional
 

In [24]:
d_master = pd.read_csv('서울시 읍면동마스터 정보.csv',encoding='cp949')
d_subway = pd.read_csv('서울시 행정동별 지하철 총 승차 승객수 정보.csv',encoding='cp949')

FileNotFoundError: [Errno 2] No such file or directory: '서울시 행정동별 지하철 총 승차 승객수 정보.csv'